# Training and Deploying Huggingface Models on Sagemaker

For this tutorial it is recommened to use 1 GPU to speed up processes, this notebooks was run the machinetype ml.p3.2xlarge.

This tutorial will focus on utilizing huggingface hub which is a repository for user to share and download machine learning models, datasets, and demos. AWS has partnerd with huggingface to allow users to utilize these resources without the need to manually create a account or token with hugging face. All resources are avaiable using the sagemaker.huggingface API.

For this tutorial we will load in a model and dataset from huggingface and train and test our model before deploying it on Sagemaker. The model we will be deploying is Flan T5 and the datasets is [ccdv/pubmed-summarization](https://huggingface.co/datasets/ccdv/pubmed-summarization).

In [ ]:
#Enable GPU to be in persistant mode
!nvidia-smi -pm 1

### Install Tools

Huggingface **transformers** are an open-source framwork that allows you to utilize APIs and tools to download pretrained models, set hyperparameters, tokenize datasets, and further tune them to suite your needs. Here we are updating Sagemaker as well as installing the transformers package and **datasets** so that we can have access to huggingface datasets and as a bonus we are adding the S3 feature to help download datasets that may already be in a S3 bucket.

In [31]:
!pip install "sagemaker" "transformers" "datasets[s3]" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.2/886.2 kB 38.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/1a/06/3817f9bb923437ead9a794f0ac0d03b8b5e0478ab112db4c413dd37c09da/transformers-4.33.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 30.0 MB/s eta 0:00:00
  Obtaining dependency information for datasets[s3] from https://files.pythonhosted.org/packages/09/7e/fd4d6441a541dba61d0acb3c1fd5df53214c2e9033854e837a99dd9e0793/datasets-2.14.5-py3-none-any.whl.metadata
  Obtaining dependency information for huggingface-hub<1.0,>=0.15.1 from https://files.pythonhosted.org/packages/72/21/51cddb8850ed3f4dbc21e57c3dabc49e64d5577857ddda7b2eb0ffc2ec0e/huggingface_hub-0.17.2-py3-none-any.whl.metadata
  Obtaining dependency information for regex!=2019.12.17 from https://files.pythonhosted.org/packages/d1/df/460ca6171a8494fcf37af43f52f6fac23e38784bb4a2

The next set of tools we will install are **sentencepiece** which is a unsupervised text tokenizer and **accelerate** another huggingface tool that allows pytorch models to run on multiple GPUs.

In [32]:
!pip install "sentencepiece" "accelerate" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.5 MB/s eta 0:00:00
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/d9/92/2d3aecf9f4a192968035880be3e2fc8b48d541c7128f7c936f430d6f96da/accelerate-0.23.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 40.7 MB/s eta 0:00:00


### Set up your Sagemaker Session

The following commands have Sagemaker create a session that will automatically create a bucket which will store our training and testing datasets, extract our role id, and region both will be used later for hypertuning.

In [1]:
import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::450119254715:role/SageMakerFullAccess
sagemaker bucket: sagemaker-us-east-1-450119254715
sagemaker session region: us-east-1


### Download your dataset from Huggingface

We will be downloading huggingface dataset 'ccdv/pubmed-summarization' which contains article titles and their abstracts which will help train our model to summarize the scientific articles. Once the dataset is loaded we'll split the data into test and train datasets.

In [34]:
from sagemaker.huggingface import HuggingFaceModel

In [85]:
from datasets import load_dataset

# load dataset
train_dataset, test_dataset = load_dataset("ccdv/pubmed-summarization", split=["train", "test"])


Now that we have our datasets we can upload our model which will be the small version of Flan T5.

**Flan T5** is a text-to-text generation model and an advancement to the original T5 model and can be run on both CPUs and GPUs. **Text-to-text** is a method of creating text by using a neural network to generate new text from a given input. These T5 models can be fine-tuned for various zero shot NLP tasks that we have seen and heard of before: text classification, summarization, translation, and question-answering. Text-to-text is not to be confused by text2text generation which is a earlier version of T5 that is designed specifically for sequence-to-sequence tasks, such as machine translation and text generation and is limited to these task where as T5 models are more flexiable due to the wider range of NPL tasks they can execute.

Because it is a seq2seq class model we will be using the transformer **AutoModelForSeq2Seq** to help find a load our pretrained model architecture. Then we will assign an **AutoTokenizer** to preprocess the text of our inputs (the test and train datasets) into an array of numbers.

In [36]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpiukgp_pa
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpiukgp_pa/_remote_module_non_scriptable.py


Now that we have loaded the architecture of our model and configured it to tokenize our inputs we can now implement a tokenization functions to start processing our datasets.

The function below will tokenize each row of our dataset based on the 'article' column. now that we have our function the next step is to implement the **map** function to interate our **tokenize function** to our loaded datasets.Then the last step will be to set our data format to be suitable for Pytorch. As you can see there are three columns represented in the dataset:
- **input_ids:** ID for each token, as each text is broken up into sequences (which can be words or subwords) and converted to tokens within our dataset they are assign an ID.
- **attention_masks:** Tokens that should be ignored by the model usually represented by a 0. Masking can be done when some sequences are not the same length so they can not belong in the same tensor and need to be padded.
- **abstracts:** The new name of the abstract column, which is the column we are implementing the new Pytorch format 

In [89]:
# create tokenization function
def tokenize(batch):
    return tokenizer(batch["article"], padding="max_length", truncation=True)


# tokenize train and test datasets
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# set dataset format for PyTorch
train_dataset =  train_dataset.rename_column("abstract", "abstracts")
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "abstracts"])
test_dataset = test_dataset.rename_column("abstract", "abstracts")
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "abstracts"])

Map:   0%|          | 0/119924 [00:00<?, ? examples/s]

### Setting up our Datasets for Training 

We are using the training script run_summarization.py which weill help train our Flan T5 model to summarize our pubmed datasets. To pass inputs to this script we first need to convert our datasets into cvs formats and then push them into our S3 bucket with the help from the boto3 package. 

In [108]:
from io import BytesIO
import boto3

#convert train dataset to csv and push to S3 bucket
csv_buffer = BytesIO()
train_dataset.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(f'{sess.default_bucket()}', 'train.csv').put(Body=csv_buffer.getvalue())

Creating CSV from Arrow format:   0%|          | 0/120 [00:00<?, ?ba/s]

TypeError: expected string or bytes-like object

In [114]:
#convert test dataset to csv and push to S3 bucket
csv_buffer = BytesIO()
test_dataset.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(f'{sess.default_bucket()}', 'test.csv').put(Body=csv_buffer.getvalue())

Creating CSV from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

{'ResponseMetadata': {'RequestId': 'JANB5YJWQB84J46C',
  'HostId': 'q3C+dMOfLMglp6NJsMWILtjntR0JhnUKQowX7JBUz155OaQPGDts/CtdJbNFeYGpDdcuoZhzHlc=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'q3C+dMOfLMglp6NJsMWILtjntR0JhnUKQowX7JBUz155OaQPGDts/CtdJbNFeYGpDdcuoZhzHlc=',
   'x-amz-request-id': 'JANB5YJWQB84J46C',
   'date': 'Thu, 21 Sep 2023 17:40:21 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"a560424683190c8e20d0aaa40857600d"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"a560424683190c8e20d0aaa40857600d"',
 'ServerSideEncryption': 'AES256'}

Here we will be saving the location of our datasets and group with a label called **data** which will be used when we execute the training of our model.

In [2]:
# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/train.csv'

# save test_dataset to s3
test_input_path = f's3://{sess.default_bucket()}/test.csv'

# Group the training and testing data since this will be the input to our hugging face estimator
data = {
    'train': training_input_path,
    'test': test_input_path
}

### Training our Model

The first step to training our model other than setting up our datasets is to set our **hyperparameters**. Hyperparameters depend on your training script and for this one we need to identify our model, the location of our train and test files, if we want to train and test our model, etc other hyperparameters are defined on the huggingface transformers Github [here](https://github.com/huggingface/transformers/tree/main/examples/pytorch/summarization).

our train andn test file locations are 'opt/ml/input/data/train' and 'opt/ml/input/data/test' because Sagemaker will train our model on a docker container and pull our files from our bucket and store then in a test and train directory. It will then output a model file back into our S3 bucket by first storing it into the directory '/opt/ml/model'.

In [3]:
#location of our run_summarization.py training file, which will be downloaded automatically
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.26.0'} # v4.6

In [4]:
# hyperparameters, which are passed into the training job
hyperparameters={'per_device_train_batch_size': 2,
                 'per_device_eval_batch_size': 4,
                 'model_name_or_path': 'google/flan-t5-small',
                 'train_file': '/opt/ml/input/data/train/train.csv',
                 'test_file':'/opt/ml/input/data/test/test.csv',
                 'text_column':'article',
                 'summary_column':'abstracts',
                 #'source_prefix': "summarize: ",
                 'do_train': True,
                 'do_eval': False,
                 'do_predict': True,
                 'predict_with_generate': True,
                 'output_dir': '/opt/ml/model',
                 'num_train_epochs': 3,
                 'learning_rate': 5e-5,
                 'seed': 7,
                 'fp16': True,
                 }

To help things move faster we will enable dataparallel which will break up the training tasks and run them in parallel. This does require we run our training on more then one instance and limits the machinetypes we can use:
ml.p3.16xlarge, ml.p3dn.24xlarge, ml.p4d.24xlarge, ml.p4de.24xlarge

**Note:** If you choose not to use the distributor commment out that line of code and the line of code that says "dictribute=distribute" in the huggingface estimator set up.

now we can set up our huggingface estimator which is a Sagemaker managed execution environment meaning the docker contianer that we mentioned before. The container will run our training script on our model utilizing a machine type of our choosing and pass our hyperparameters to the container as well. In the end the estimator will create a huggingface directory in the default bucket and output a model.tar.gz file which we can deploy to a endpoint.

In [40]:
# configuration for running training on smdistributed Data Parallel
distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

from sagemaker.huggingface import HuggingFace

# create the Estimator
huggingface_estimator = HuggingFace(
      entry_point='run_summarization.py', # script
      source_dir='./examples/pytorch/summarization', # relative path to example
      git_config=git_config,
      instance_type='ml.p4d.24xlarge',
      checkpoint_s3_uri=f's3://{sess.default_bucket()}/checkpoints1',
      #checkpoint_local_path='/opt/ml/checkpoints',
      instance_count=2,
      transformers_version='4.26.0',
      pytorch_version='1.13.1',
      py_version='py39',
      role=role,
      hyperparameters = hyperparameters,
      distribution = distribution
)

Start the training which we can also monitor and view logs on the console by going to `Sagemaker > Training > Training Jobs.`

**Warning:** If you recieve a **ResourceLimitExceeded** error it's because there are not enough resources on AWS to use this instance at the moment. To solve this error either try another instance type or try running the training again to see if resources have become available.

In [42]:
# starting the train job
huggingface_estimator.fit(data)

Cloning into '/tmp/tmpq0n_g128'...
Note: switching to 'v4.26.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 820c46a70 Hotifx remove tuple for git config image processor. (#21278)
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2023-09-22-11-11-50-879


Using provided s3_resource


ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTrainingJob operation: The account-level service limit 'ml.p4d.24xlarge for training job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 2 Instances. Please use AWS Service Quotas to request an increase for this quota. If AWS Service Quotas is not available, contact AWS support to request an increase for this quota.

### Deploy the Model

Here we are creating a endpoint and deploying our model to said endpoint the next step will be to feed the model some inputs and check that it produces a accurate and consise summary.

We are deploying our enpoint using 1 GPU which can take 20min to run, feel free to try out other machine types that utilize more GPUs.

In [5]:
predictor = huggingface_estimator.deploy(initial_instance_count=1, instance_type="ml.g4dn.xlarge")

SyntaxError: positional argument follows keyword argument (3585978380.py, line 1)

**Optional** 

If your model takes a long time to train you can come back to your notebook later without worrying about it stopping. If you choose to do this the following code is another way to obtain our model.tar.gz file from your bucket and deploy it. Remember you can monitor your training on the console by going to `Sagemaker > Training > Training Jobs.`

Sometimes you need to search in your default bucket to look for your model.tar.gz file it will be in one of the directories that says 'huggingface-pytorch-training'. We are deploying our enpoint using 1 GPU which can take 20min to run, feel free to try out other machine types that utilize more GPUs.

In [44]:
from sagemaker.huggingface.model import HuggingFaceModel
#huggingface directory that holds your model.tar.gz file
huggingface_directory= "<enter in the directory name>"

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=f's3://{sess.default_bucket()}/{huggingface_directory}/output/model.tar.gz',  # path to your trained SageMaker model
   role=role,                                            # IAM role with permissions to create an endpoint
   transformers_version="4.26",                           # Transformers version used
   pytorch_version="1.13",                                # PyTorch version used
   py_version='py39',                                    # Python version used
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.g4dn.xlarge"
)

INFO:sagemaker:Creating model with name: huggingface-pytorch-inference-2023-09-22-11-17-19-541
INFO:sagemaker:Creating endpoint-config with name huggingface-pytorch-inference-2023-09-22-11-17-20-256
INFO:sagemaker:Creating endpoint with name huggingface-pytorch-inference-2023-09-22-11-17-20-256


---------------------------------------------------------------------!

### Submit Inputs and Parameters to the Model

Now we can pass in fomr text for our model to summarize. Below you will see that we have provided a paragraph about SARS-CoV-2 as our prompt, we also have some parameters that we specify to further tune our model to get a consise summary of what our prompt is about.

- **Max_Length:** Max number of words to generate.
- **Num_Return_Sequences:** Number of different outputs to generate. For our example we want one sentence or sequence.
- **Temperature:** Controls randomness, higher values increase diversity meaning a more unique response make the model to think harder. Must be a number from 0 to 1.
- **Top_p (nucleus):** The cumulative probability cutoff for token selection. Lower values mean sampling from a smaller, more top-weighted nucleus. Must be a number from 0 to 1.
- **Top_k**: Sample from the k most likely next tokens at each step. Lower k focuses on higher probability tokens.This means the model choses the most probable words. Lower values eliminate fewer coherent words.

In [67]:
prompt =  """Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) is a
highly transmissible and pathogenic coronavirus that emerged in late 2019 and has 
caused a pandemic of acute respiratory disease, named ‘coronavirus disease 2019’ (COVID-19), 
which threatens human health and public safety. In this Review, we describe the basic virology of 
SARS-CoV-2, including genomic characteristics and receptor use, highlighting its key difference 
from previously known coronaviruses. We summarize current knowledge of clinical, epidemiological and 
pathological features of COVID-19, as well as recent progress in animal models and antiviral treatment 
approaches for SARS-CoV-2 infection. We also discuss the potential wildlife hosts and zoonotic origin 
of this emerging virus in detail."""

payload = ({
    "inputs": prompt,
    "parameters": {"max_length": 2000, 
                   "num_return_sequences": 1, 
                   "temperature":0.6,
                   "top_k": 50, 
                   "top_p": 0.95,
                   "do_sample": True,
                  }
})
predictor.predict(payload)

[{'generated_text': 'In this review, we summarize current knowledge of clinical, epidemiological and pathological features of SARS-CoV-2 and discuss recent progress in animal models and antiviral treatment approaches for SARS-CoV-2 infection.'}]

**Warning:** Once you are done don't forget to delete your endpoint, model, buckets, and shutdown or delete your Sagemaker notebook to avoid additional charges!

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()